In [1]:
import tqdm
import datetime
import argparse
import numpy as np
from typing import Dict
from pathlib import Path
from sklearn.metrics import accuracy_score, balanced_accuracy_score

import wandb
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

from fa.utils import AttrDict, save_checkpoint, seed_everything
from fa.dgcnn import DGCNNSeg
from fa.fusion import SimpleAggregator
from fa.predictor import DGCNNPredHead
from fa.model import FindAnything
from fa.dataset import FindAnythingDataset
from fa.eval_utils import compute_iou, get_pc_viz
from fa.train import evaluate_model

In [2]:
config = AttrDict()

# Setup
config.seed = 0
config.device = "cuda"
config.num_workers = 1
config.checkpoint = "mvp_poster"

# Model
config.aggr_feat_size = 128

# Test
config.test_dataset_size = 1500
config.batch_size = 1
config.pos_sample_weight = 2
config.pred_threshold = 0.5

# Problem Framework
config.num_query_points = 4096
config.num_support_points = 1024

In [3]:
test_dataset = FindAnythingDataset(
        split="test",
        num_scene_points=config.num_query_points,
        num_template_points=config.num_support_points,
        dataset_size=config.test_dataset_size
    )

test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=config.batch_size,
    shuffle=False,
    num_workers=config.num_workers,
)

In [4]:
checkpoint = torch.load(Path("checkpoints") / config.checkpoint / "last_epoch")
model = checkpoint["model"]
model.to(config.device)

FindAnything(
  (scene_feat_extractor): DGCNNSeg(
    (conv1): Sequential(
      (0): Conv2d(12, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (conv2): Sequential(
      (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (conv3): Sequential(
      (0): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (conv4): Sequential(
      (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2):

In [5]:
metrics = evaluate_model(model, test_dataloader, config, generate_visuals=False)

Test:   0%|          | 0/1500 [00:00<?, ?it/s]


RuntimeError: The size of tensor a (4096) must match the size of tensor b (1024) at non-singleton dimension 1

In [ ]:
for k, v in metrics.items():
    print(f"{k}: {v}")